                                                  Assignment 3
210100111                                                                                 Prachit Gupta

In [245]:
import numpy as np
from scipy.linalg import polar

In [246]:
### input data
##define matrix
V = np.array([[0.63,0.45,0.87],
              [0.03,0.40,0.86],
              [0.86,0.17,0.16],
              [0.29,0.79,0.04],
              [0.60,0.38,0.15],
              [0.99,0.43,0.76]])

W = np.array([[0.17,0.03,0.50],
              [0.26,0.37,0.15],
              [0.78,0.65,0.67],
              [0.83,0.96,0.82],
              [0.69,0.72,0.04],
              [0.64,0.77,0.34]])

In [247]:
## obtain matrix from quaternion
def quaternion_to_dcm(q):
    q1, q2, q3, q0 = q
    dcmT = np.array([[q0**2 + q1**2 - q2**2 - q3**2, 2*(q1*q2 - q0*q3), 2*(q1*q3 + q0*q2)],
                     [2*(q1*q2 + q0*q3), q0**2 - q1**2 + q2**2 - q3**2, 2*(q2*q3 - q0*q1)],
                     [2*(q1*q3 - q0*q2), 2*(q2*q3 + q0*q1), q0**2 - q1**2 - q2**2 + q3**2]])
    return dcmT.T

In [248]:
##whabhas
def RotationMatrix(v,w):
    ##following steps acc to whabhas algorithm
    ## M = Ux(N.T)x(X.T)xN
    ##compute square matrix A 
    A = np.matmul(v,w.T)
    ##decomposition of A into U and P
    U, P = polar(A)
    ##compute N and d from positive definate matrix P
    eigValues,N = np.linalg.eig(P)
    d = np.diag(eigValues)
    ###compute X depending on detU
    detU =  np.linalg.det(U)
    k = len(eigValues)
    epsilon = 0.1
    if detU > -1 - 0.1 and detU < -1 + 0.1:
        print("detU is -1")
        X = np.zeros((k,k))
        X[0:k-1, 0:k-1] = np.eye(k-1)
        X[-1,-1] = -1
    elif detU > 1 - 0.1 and detU < 1 + 0.1:
        print("detU is 1")
        X = np.eye(k) 
    else:  
        print("error U is not orthogonal")
    ##compute M using M = Ux(N.T)x(X.T)xN
    M = np.matmul(np.matmul(U,(N.T),(X.T)), N)
    return M

M_whabha = RotationMatrix(V.T,W.T)
print(M_whabha)

detU is 1
[[ 0.97307269  0.22659041 -0.042265  ]
 [-0.21660168  0.96160772  0.16850611]
 [ 0.07882421 -0.15481402  0.98479407]]


In [249]:
##quest reference 1981 Shushter paper (referred equations)

def quest(V,W,weights): ##input matrix (3 x n) and weights(Summation weights = 1)
    
    ##compute B (intiution : stores input data info)
    # Repeat weights along the second axis to match the shape of W
    a_repeated = np.repeat(weights[:, np.newaxis], 3, axis=1) ## 6 x 3
    # Perform element-wise multiplication of W and the repeated weights
    W_weighted =  a_repeated.T*W 
    # Calculate the weighted sum of outer products of w and v
    B = np.transpose(np.sum(np.matmul(W_weighted, V.T), axis=0)) ##eq 48
    ##compute Z (used to compute gain K in bilinear form)
    Z = np.array([B[1, 2] - B[2, 1], B[2, 0] - B[0, 2], B[0, 1] - B[1, 0]])  # Eqn.46
    
    ##compute S
    S = B + B .T ##eq 45
    ##eq 44 params
    sig = np.trace(B) ##eq44 (Also 1/2 tarce(S))
    delta = np.linalg.det(S) ##eq63
    k = np.trace(delta * np.linalg.inv(S))##eq63
    # Calculate parameters from equation 71
    a = sig ** 2 - k
    b = sig ** 2 + np.dot(Z.T, Z)
    c = delta + np.dot(Z.T, np.dot(S, Z))
    d = np.dot(Z.T, np.dot(S ** 2, Z))
    constant = a*b + c*sig - d
    
    ##charateristics equation from 70
    Lambda = np.sum(weights) ##unity
    last_lambda = 0
    epsilon = 10e-5##high accuracy as already have estimate
    ##compute lambda using iterative newton raphson
    while abs(Lambda - last_lambda) >= epsilon:
        last_lambda = Lambda
        f = Lambda**4 - (a + b)*Lambda**2 - c*Lambda + constant
        f_dot = 4*Lambda**3 - 2*(a + b)*Lambda - c
        Lambda = Lambda - f/f_dot
      
    ##eq 66 parameters
    omega =  Lambda ## optimal
    alpha = omega**2 - sig**2 + k
    beta  = omega - sig
    gamma = (omega + sig)*alpha - delta
    
    ##optimal quaternion from eq 68 and 69
    X = np.matmul((alpha*np.eye(3) + beta*S + S@S) , Z)
    Q_opt = np.concatenate([X, [gamma]]) / np.sqrt(gamma**2 + (np.linalg.norm(X))**2)
    
    ##generate optimal rotation matrix from quaternions
    M = quaternion_to_dcm(Q_opt)
    
    return Q_opt, M

a = 1/6*np.ones((6,1)) ## weights given
Q, Mquest = quest(V.T,W.T,a) ##generate optimal matrix using quest

print(f"Quaternion estimated = {Q}\n")
print(f"Rotation matrix:\n{Mquest}\n")


Quaternion estimated = [0.08101742 0.03002042 0.11149527 0.9900019 ]

Rotation matrix:
[[ 0.97333516  0.22562541 -0.04137442]
 [-0.2158967   0.96200996  0.16710907]
 [ 0.07750666 -0.15372053  0.9850699 ]]



In [250]:
##triad

def triad_algorithm(V1, V2, W1, W2):
    # Normalize vectors
    ## known fixed vectors
    V1 = V1 / np.linalg.norm(V1) ##unit vector
    V2 = V2 / np.linalg.norm(V2)
    ##observations
    W1 = W1 / np.linalg.norm(W1)
    W2 = W2 / np.linalg.norm(W2)
    
    # Calculate reference vectors triad
    r1 = V1
    
    r2 = np.cross(V1, V2)
    r2 /= np.linalg.norm(r2)
    
    r3 = np.cross(r1,r2)
    ## reference matrix
    Mref = np.array([r1,r2,r3]).T
    print(f"reference = {Mref}\n")
    
    ##Calculate observation vectors triad
    s1 = W1
    
    s2 = np.cross(W1, W2)
    s2 /= np.linalg.norm(s2)
    
    s3 = np.cross(s1,s2)
    ##observation matrix
    Mobs = (np.array([s1,s2,s3])).T
    print(f"observation = {Mobs}\n")
    
    ##rotation matrix of transformation
    M = np.matmul(Mobs,Mref.T)
    
    return M

v1, v2 = V[0:2, :]  ##first two fixed vectors
w1,w2 = W[0:2,:]    ## first two measurements

Mtriad = triad_algorithm(v1,v2, w1, w2)

print(f"Rotation matrix: {Mtriad}\n")


reference = [[ 0.5409569   0.06847907  0.83825786]
 [ 0.38639779 -0.90550396 -0.17538337]
 [ 0.74703573  0.41877583 -0.51629878]]

observation = [[ 0.3213846  -0.83672029 -0.44340849]
 [ 0.05671493  0.48441701 -0.87299689]
 [ 0.94524883  0.25541988  0.2031387 ]]

Rotation matrix: [[-0.25513326  0.95960231  0.11861881]
 [-0.66794375 -0.26361786  0.69595744]
 [ 0.69911241  0.0983312   0.70821805]]



In [251]:
##erros and visulization
def check_error(V,W,M,output,weights):
    ## propagate weihts for element multiplication
    weights = (np.repeat(weights[:, np.newaxis], 3, axis=1)) ## 6 x1 to 3 x 6
    Vtransformed = np.matmul(M,V)
    if output == 0:
        error = np.linalg.norm(W - Vtransformed, ord=2)  # Frobenius norm
    elif output == 1:
        error = 0.5*np.sum((weights.T * np.square(np.abs(W - Vtransformed))))  # 2-norm
    return error

def display_errors(M):
    errors = []
    for i in range(V.shape[0]):
        ei = check_error(V[i,:].T,W[i,:].T,M,0,a) ##compute indivisual errors
        errors.append(ei)
    print(f"Ecludian errors is estimating each point is {errors}")    
    print(f"net minimized LOSS Function norm obtained = {check_error(V.T, W.T, M,1,a)}\n")
 
##whabha estimation
print("Whabha\n")
display_errors(M_whabha)   
##Triad estimation
print("Triad\n")
display_errors(Mtriad)   
##Quest estimation
print("Quest\n")
display_errors(Mquest)
    

Whabha

Ecludian errors is estimating each point is [0.736319097058084, 0.6788349921329151, 0.8042195665622492, 0.9886861265907655, 0.46997946853176503, 0.7235458800089997]
net minimized LOSS Function norm obtained = 0.280970955725879

Triad

Ecludian errors is estimating each point is [0.6356416904093726, 0.5726922151980405, 1.4186104357377214, 1.4356826420967794, 1.316230907375301, 1.432973122256347]
net minimized LOSS Function norm obtained = 0.7159610599813325

Quest

Ecludian errors is estimating each point is [0.7362929016145645, 0.6791017162472309, 0.8044092527985307, 0.9883707392395488, 0.46955757081373584, 0.7235809751751199]
net minimized LOSS Function norm obtained = 0.2809425901745521



Comments : Rotation matrix for estimation are quite similar for Quest and Whabhas Algorithm which minimizes loss function to a significant extend

Triad method for estimation is computationally relatively simple but giving a prediction which are way off from reference vectors 

